In [ ]:
get_ipython().run_line_magic('config', 'IPCompleter.greedy=True')
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
from matplotlib import style
style.use('seaborn-whitegrid')

get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns
import numpy as np

import sys
import os

 ### Read data

In [ ]:
df_clean = pd.read_pickle(os.path.join('..', 'data/despesas56_clean.pkl'))
df_clean.head()

 ### Fit data do analysis

In [ ]:
df = df_clean.copy()

# switch full date to month-year and drop full date
df['periodo'] = df['dataGasto'].dt.to_period('M')

#select year period
df = df.loc()[(df.periodo > '2019-06') & (df.periodo <= '2020-06')]

#remove compensatory vaues
df = df[~df['numDocumento'].str.match('Comp')].reset_index(drop=True)


 ### Expenses by month

In [ ]:
fig, ax = plt.subplots(figsize=(18,7))
df.groupby('dataGasto')['valor'].sum().plot(ax=ax)


 ### Expenditure by Education Level

In [ ]:
gasto_escolaridade = df[['nome','escolaridade', 'valor']].copy()

data_na_index = gasto_escolaridade[gasto_escolaridade['escolaridade'].isnull()].index

gasto_escolaridade.drop(data_na_index, axis=0, inplace=True)

n_por_escolaridade = gasto_escolaridade[['escolaridade', 'nome']].groupby('escolaridade')['nome'].nunique()
gasto_por_escolaridade = gasto_escolaridade[['escolaridade', 'valor']].groupby('escolaridade')['valor'].sum()

media_escolaridade = {'escolaridade': [], 'gastoMedio': []}
for escolaridade, valor in gasto_por_escolaridade.items():
    media_escolaridade['escolaridade'].append(escolaridade)
    media_escolaridade['gastoMedio'].append(int(valor / n_por_escolaridade[escolaridade]))

media_escolaridade = pd.DataFrame(media_escolaridade)

fig, ax = plt.subplots(figsize=(18,7))
ax.tick_params(axis='x', rotation=50)
sns.barplot(ax = ax, x="escolaridade", y="gastoMedio", data=media_escolaridade.sort_values(by='gastoMedio'))


 ### Heat Map plane ticket value x month x uf district

In [ ]:
# features = ['ufNascimento', 'tipo', 'fornecedor', 'dataGasto', 'valor']
features = ['ufNascimento', 'dataGasto', 'valor', 'tipo', 'fornecedor']
index_heatmap_passage = df.tipo.str.match('PASSAGEM')
df_heatmap = df[features].loc()[index_heatmap_passage].reset_index(drop=True).copy()

df_heatmap['periodo'] = df_heatmap['dataGasto'].dt.to_period('M')
df_heatmap.drop('dataGasto', axis=1, inplace=True)
df_heatmap.set_index('periodo', inplace=True)

d1 = pd.pivot_table(df_heatmap, index="periodo", columns="ufNascimento", values="valor", aggfunc=np.mean, fill_value=0)

estados = ['RS', 'SC', 'PR',
           'SP', 'MG', 'RJ', 'ES',
           'MS', 'GO', 'DF', 'MT',
           'BA', 'SE', 'AL', 'PE', 'PB', 'RN', 'CE', 'PI', 'MA',
           'TO', 'PA', 'AP', 'RO', 'AC', 'AM', 'RR']

fig, ax = plt.subplots(figsize=(18,7))
sns.heatmap(ax = ax, data=d1[estados].div(max(d1.max())))


 ### Bar plot by Partido

In [ ]:
features = ['partido', 'sexo']
df_sex_proportion = df.drop_duplicates(subset='nome')[features]

df_sex_proportion['sexo'] = df_sex_proportion['sexo'].str.replace(pat='M', repl='Homem')
df_sex_proportion['sexo'] = df_sex_proportion['sexo'].str.replace(pat='F', repl='Mulher')

df_sex_proportion = df_sex_proportion.groupby(['partido', 'sexo'])['partido'].count().unstack(fill_value=0)

plt.figure()
df_sex_proportion.plot(kind='bar',stacked=True, figsize=(18,8))
plt.legend(fontsize=20)



 ### Bar plot by UF

In [ ]:
features = ['ufPartido', 'sexo']
df_sex_proportion = df.drop_duplicates(subset='nome')[features]

df_sex_proportion['sexo'] = df_sex_proportion['sexo'].str.replace(pat='M', repl='Homem')
df_sex_proportion['sexo'] = df_sex_proportion['sexo'].str.replace(pat='F', repl='Mulher')

df_sex_proportion = df_sex_proportion.groupby(['ufPartido', 'sexo'])['ufPartido'].count().unstack(fill_value=0)

plt.figure()
df_sex_proportion.plot(kind='bar',stacked=True, figsize=(18,8))
plt.legend(fontsize=20)



### Relation between type of expenses and document emmited

In [ ]:
features = ['tipo', 'tipoDocumento', 'valor']

df_tipo = df[features].copy()

plt.figure(figsize=(10,5))
chart = sns.catplot(data=df_tipo, x='tipo', y = 'valor', hue='tipoDocumento', s = 4, jitter=True, height=7, aspect=2)
chart.set_xticklabels(rotation=70, horizontalalignment='right')

In [ ]:
summary = df.groupby(['nome', 'mes'])['valor'].sum().unstack().copy()
summary['average'] = summary.mean(numeric_only=True, axis=1)
summary.sort_values(by='average')